In [1]:
#Load the dependancies
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
from fastai.medical.imaging import *
from matplotlib import pyplot as plt
import pydicom
import itk
from pathlib import Path
import os

In [10]:
rootdir = "C:/Users/Visagio/Desktop/dt/datasets/"
_, dirs, _ = next(os.walk(rootdir))

In [2]:
#Initializing ITK readers
def initReaders():
    reader = itk.ImageFileReader[itk.Image[itk.SS,3]].New()
    atlasReader = itk.ImageFileReader[itk.Image[itk.SS,3]].New()
    labelReader = itk.ImageFileReader[itk.Image[itk.UC,3]].New()
    return reader, atlasReader, labelReader

In [8]:
#Set file names for readers
def setReadersParameters(examNumber, reader, atlasReader, labelReader):
    reader.SetFileName(f'C:/Users/Visagio/Desktop/dt/nifti-output/{examNumber}.nii.gz')
    atlasReader.SetFileName('C:/Users/Visagio/Desktop/dt/atlasImage.mha')
    labelReader.SetFileName('C:/Users/Visagio/Desktop/dt/atlasMask.mha')
    reader.Update()
    atlasReader.Update()
    labelReader.Update()


In [4]:
# #Set file names for readers
# reader.SetFileName('C:/Users/Visagio/Desktop/dt/nifti/6_AX_FLAIR_20161004105456_6000.nii')
# atlasReader.SetFileName('C:/Users/Visagio/Desktop/dt/atlasImage.mha')
# labelReader.SetFileName('C:/Users/Visagio/Desktop/dt/atlasMask.mha')
# reader.Update()
# atlasReader.Update()
# labelReader.Update()

In [5]:
#Create Skull Stripping filter
def createSkullStrippingFilter(reader, atlasReader, labelReader):
    filter = itk.StripTsImageFilter[itk.Image[itk.SS,3], itk.Image[itk.SS,3]].New()
    filter.SetInput(reader.GetOutput())
    filter.SetAtlasImage(atlasReader.GetOutput())
    filter.SetAtlasBrainMask(labelReader.GetOutput())
    filter.Update()
    return filter

In [6]:
#Creating Mask Filter
def createMaskFilter(reader, filter):
    maskFilter = itk.MaskImageFilter[itk.Image[itk.SS,3], itk.Image[itk.UC,3], itk.Image[itk.SS,3]].New()
    maskFilter.SetInput1(reader.GetOutput())
    maskFilter.SetInput2(filter.GetOutput())
    maskFilter.Update()
    maskFilter.GetOutput()
    return maskFilter

In [14]:
def writeMaskAndMaskedImageToFiles(examNumber, filter, maskFilter):
    Path(f"C:/Users/Visagio/Desktop/dt/skullstripped/{examNumber}").mkdir(exist_ok=True)
    #Creating MaskWriter
    maskWriter = itk.ImageFileWriter[itk.Image[itk.UC,3]].New()
    maskWriter.SetInput(filter.GetOutput())
    maskWriter.SetFileName(f'C:/Users/Visagio/Desktop/dt/skullstripped/{examNumber}/mask.nii.gz')

    #Creating ImageWriter
    imageWriter = itk.ImageFileWriter[itk.Image[itk.SS,3]].New()
    imageWriter.SetInput(maskFilter.GetOutput())
    imageWriter.SetFileName(f'C:/Users/Visagio/Desktop/dt/skullstripped/{examNumber}/masked.nii.gz')

    maskWriter.Update()
    imageWriter.Update()

In [15]:
for examNumber in dirs:
    reader, atlasReader, labelReader = initReaders()
    setReadersParameters(examNumber, reader, atlasReader, labelReader)
    filter = createSkullStrippingFilter(reader, atlasReader, labelReader)
    maskFilter = createMaskFilter(reader, filter)
    writeMaskAndMaskedImageToFiles(examNumber, filter, maskFilter)